# 各种常用sklearn算法实践特性分析

# 1. LR

## 特点分析

- 优点：
    - 实现简单，需要配置的超参数少
    - 训练速度快，支持多线程
- 缺点：
    - 模型的复杂度低，且无法通过超参数增大复杂度
    - 对复杂的分类问题，很难达到非常高的分类性能
    
LR非常适合在不做任何配置的情况下做**baseline**，其他的复杂模型与LR baseline性能相比的增益才是有意义的。 

## solver选择

LR默认的优化求解器，不适用于大数据集，需要手动设置为solver='lbfgs'。
不同solver的速度差别：

- Liblinear，当前的默认solver，不支持多线程
- lbfgs，后续版本的默认solver

|训练数据量| Liblinear  | lbfgs |
|--|--|--|
| 1w | 6.97 | 9.3 |
|10w| 105s | 7.8s|

在数据量小的时候，可以使用默认的Liblinear，一旦数据量增大，必须切换到lbfgs，才能快速求解。



In [ ]:
from sklearn.linear_model import LogisticRegression
clf_lr = LogisticRegression(random_state = 0, verbose=1, solver='lbfgs', n_jobs=-1)

# 交叉验证方法

K折交叉验证的优势：

- 在没有测试集GT的情况下，可以更准确地评估模型的性能
- 避免手动切分训练集和验证集，造成对训练集过拟合
- 对k折的test predict结果去平均作为最终输出

实现方式：

- 用`KFold`划分训练集和验证集的index，手动写循环
可以在循环中依次predict test结果，得到均值作为输出
- 如果只需要模型在验证集上的score，可以直接用cross_val_score实现

In [ ]:
# KFold实现
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
# Create the kfold object
k_fold = KFold(n_splits = 3, shuffle = True, random_state = 50)

def run_cv(clf):
    '''
    - clf，典型的sklearn分类器，具有fit/predict_proba 方法
    - submission，输出的k折平均的测试集结果
    '''
    # 用于记录各轮的测试集分数,验证集score
    test_prob = np.zeros(test_fill.shape[0])
    val_scores = []
    
    for idx_train, idx_val in k_fold.split(train):
        X_train, y_train = train[idx_train], train_labels[idx_train]
        X_val, y_val = train[idx_val], train_labels[idx_val]
        clf.fit(X_train, y_train)
        val_score = roc_auc_score(y_val, clf.predict_proba(X_val)[:, 1])
        val_scores.append(val_score)
        test_prob += model.predict_proba(test_data)[:, 1] / k_fold.n_splits

    submission = pd.DataFrame({'ID': test_ids, 'TARGET': test_prob})
    print('mean val score: {}'.format(np.mean(val_scores)))
    return submission

submission = run_cv(clf)

# 用StratifiedKFold 实现
from sklearn.model_selection import StratifiedKFold
kfold = StratifiedKFold(n_splits=10,
                        random_state=1).split(X_train, y_train)
scores = []
for k, (train, test) in enumerate(kfold):
    pipe_lr.fit(X_train[train], y_train[train])
    score = pipe_lr.score(X_train[test], y_train[test])
    


In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(estimator=pipe_lr,
                         X=X_train,
                         y=y_train,
                         cv=10,
                         n_jobs=1)
# 返回的 scores 是各类val分数的list
print('CV accuracy: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

# Baseline 性能对比

HomeCredit，用特征工程之后的数据，评测lgb最优模型与其他baseline分类器的性能差异。

数据说明：

- data_fe，特征工程生成的数据
- data_fill，填充null，处理极大值之后的数据

由于LGB可以直接输入带null的数据，而其他分类器需要对填充null，故统一用填充处理后的data_fill评测。

评测结果：各个分类器的auc指标很低，最高仅为0.584。而使用data_fe训练的val auc在0.78。

|    clf      |   test_score |   val_score |
|:------------|-------------:|------------:|
| lgbm_tune   |    nan       |    0.58447  |
| lgbm_base   |    nan       |    0.5642   |
| xgb         |    nan       |    0.54     |
| lr_baseline |      0.54191 |    0.516605 |
| mlp_32      |      0.5284  |    0.5142   |


观点：

- 强制imputer可能给数据带来了不利的噪音，例如大量的null用median填充，对于null比例很高的col，不合理
- 总体上，lgb的学习能力是强于其他model。

数据imputer的方法如下：

In [ ]:
def impute_inf(df_data):
    imputer = Imputer(strategy = 'median')
    # replace inf as nan
    df_data.replace([np.inf, -np.inf], np.nan, inplace=True)
    # impute nan to median
    df_data = imputer.fit_transform(df_data)
    return df_data

train_fill = impute_inf(train)

# lgb

lgb对nan数据处理的机制是什么

